In [48]:
print("OK")

OK


In [4]:
%pwd

'c:\\Users\\jisna\\medical_chatbot\\End-to-End-Medical-Chatbot'

In [2]:
import os
os.chdir('../')


In [5]:
%pwd

'c:\\Users\\jisna\\medical_chatbot\\End-to-End-Medical-Chatbot'

In [6]:


from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [55]:
from langchain.llms import CTransformers

In [57]:
PINECONE_API_KEY="pcsk_74RJkT_TFVcRGZFrzG69LPdMqdcWg9pPqHzuLw6vvTRPxjv5aDGR8Teqw1SzYQX67BgLxe"
PINECONE_API_ENV="us-east-1"

In [7]:
#Extract data from the pdf
def load_pdf(data):
     loader= DirectoryLoader(data,glob="*.pdf",
                    loader_cls=PyPDFLoader)
     documents=loader.load()
     return documents

     

In [8]:
extracted_data=load_pdf("data/")

In [9]:
# Create text chunks from loaded data
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    chunks=text_splitter.split_documents(extracted_data)
    return chunks

In [10]:
text_chunks=text_split(extracted_data)
print(f"Length of chunks: {len(text_chunks)}")

Length of chunks: 5860


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
def download_hugging_face_embedding():
    embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [13]:
embeddings=download_hugging_face_embedding()

C:\Users\jisna\AppData\Local\Temp\ipykernel_134616\22862419.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\jisna\anaconda3\envs\mchatbotnew\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result=embeddings.embed_query("Hello World")
print(f"Length of query result: {len(query_result)}")

Length of query result: 384


In [16]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")

In [18]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
pc=Pinecone(api_key=PINECONE_API_KEY)
index_name="mchatbot"

In [19]:
if index_name not in pc.list_indexes().names():
   pc.create_index(
       name=index_name,
       dimensions=384,
       metric='cosine',
       spec=ServerlessSpec(cloud='aws',region='us-east-1')

 )

In [78]:
index=pc.Index(index_name)
index

In [21]:
import os
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [22]:
#Embed each chunk and upsert the embeddings into your Pinecone Index
from langchain_pinecone import PineconeVectorStore
doc_search=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [83]:
doc_search

In [23]:
#load from existing index
doc_search=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [37]:
retriever=doc_search.as_retriever(
                 search_type='similarity',search_kwargs={"k":3}

)

In [30]:
retrieved_docs=retriever.invoke("What is Acne?")

In [31]:
retrieved_docs

[Document(id='aea72780-d284-4e41-a888-00a471af9b82', metadata={'page': 39.0, 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='8309ffe4-f26b-4ea0-b575-e531c6b2545d', metadata={'page': 39.0, 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='aca5a49c-4385-4396-ad24-bca6c7a7fce2', metadata={'page': 38.0, 'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='f988984f-5272-4966-bd64-ae4062008619', metadata={'page': 38.0, 'source': 'data\\Medical_bo

In [32]:
from langchain.llms import CTransformers

In [33]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens': 500,
                          'temperature':0.4})

In [46]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an intelligent assistant specializing in answering questions based on provided context. "
    "Carefully read the retrieved docs and provide a concise, accurate response. "
    "If the retrieved docs do not contain the information needed to answer, you MUST respond with 'I don't know.' "
    "Do not attempt to predict or fabricate information. "
    "Keep your response to three sentences or fewer to maintain brevity."
    "\n\n"
    "Context:\n{context}"
)


Human_prompt="{input}"
prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human",Human_prompt)
    ]
)

In [47]:
question_answer_chain=create_stuff_documents_chain(llm,prompt)

In [48]:
rag_chain=create_retrieval_chain(retriever,question_answer_chain)

In [43]:
response=rag_chain.invoke({"input":"What is Acne ?"})
print(response['answer'])


Assistant: According to the Gale Encyclopedia of Medicine, acne is a skin disorder that occurs when the sebaceous glands become inflamed. It can affect anyone but is more common in women during puberty and in menstruating women. The exact cause of acne is not known, but it is believed to be related to hormonal changes in the body. (Sources: Gale Encyclopedia of Medicine, 2nd ed.)


In [ ]:
%huggingface-cli login

In [49]:
response=rag_chain.invoke({"input":"Explain Stats?"})
print(response['answer'])

Number of tokens (513) exceeded maximum context length (512).
Number of tokens (514) exceeded maximum context length (512).
Number of tokens (515) exceeded maximum context length (512).
Number of tokens (516) exceeded maximum context length (512).
Number of tokens (517) exceeded maximum context length (512).
Number of tokens (518) exceeded maximum context length (512).
Number of tokens (519) exceeded maximum context length (512).
Number of tokens (520) exceeded maximum context length (512).
Number of tokens (521) exceeded maximum context length (512).
Number of tokens (522) exceeded maximum context length (512).
Number of tokens (523) exceeded maximum context length (512).
Number of tokens (524) exceeded maximum context length (512).
Number of tokens (525) exceeded maximum context length (512).
Number of tokens (526) exceeded maximum context length (512).
Number of tokens (527) exceeded maximum context length (512).
Number of tokens (528) exceeded maximum context length (512).
Number o


```
I'm happy to help! A complete blood count (CBC) is a series of tests that evaluate the different types of cells in your blood, including red blood cells, white blood cells, and platelets. It's often performed as part of a routine physical examination and can provide valuable information about the health of your blood and other body systems. Abnormal results can indicate the presence of abnormal results can indicate the result indicates abnormal results can indicate the CBC results can indications inormal results can indicate the results can indicate the results can indicate the results can indicate the presence of abnormal results can indicate the results can indicate the test results can indicate the results can indicate the results can indicate the CBC results can indicate the presence of abnormal results can indicate the results can indicate the results can indicate the results can indicate the results can indicate the results can indicate the results can indicate the results c

In [151]:
response=rag_chain.invoke({"input":"what is math?"})
print(response['answer'])

Number of tokens (513) exceeded maximum context length (512).
Number of tokens (514) exceeded maximum context length (512).
Number of tokens (515) exceeded maximum context length (512).
Number of tokens (516) exceeded maximum context length (512).
Number of tokens (517) exceeded maximum context length (512).
Number of tokens (518) exceeded maximum context length (512).
Number of tokens (519) exceeded maximum context length (512).
Number of tokens (520) exceeded maximum context length (512).
Number of tokens (521) exceeded maximum context length (512).
Number of tokens (522) exceeded maximum context length (512).
Number of tokens (523) exceeded maximum context length (512).
Number of tokens (524) exceeded maximum context length (512).
Number of tokens (525) exceeded maximum context length (512).
Number of tokens (526) exceeded maximum context length (512).
Number of tokens (527) exceeded maximum context length (512).
Number of tokens (528) exceeded maximum context length (512).
Number o



Response:
Math is a subject refers to humans, or mathematics is a fundamental concepts and arth is a fundamental to human is a fundamental concepts are numbers, short for humans, in simple definition: Math is a fundamental to humans, or mathematics is a fundamental concepts include: Math is a branch of the term: Math is a subject is a subject is a subject dealsays (short for humans refers to humans? Oh, as an abstractly is a subject refers to human is a subject is a fundamental to humans, short for humans refers to human refers to a subject refers to a subject refers to a subject is a subject is a subject is a subject involves the branch of coursework is a field covers variously is a subject deals are a subject refers to a subject refers to a fundamental concepts and science deals are numbers, or mathematics is a fundamental concepts can be a fundamental to humans? Oh!  is a fundamental concepts include: Math is a fundamental concepts referstopper—a is a language for human
is a subje